In [7]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python

In [8]:
from subprocess import check_output
print(check_output(["ls", "./export/census_tracts_lookup"]).decode("utf8"))

Alabama.csv
Alaska.csv
Arizona.csv
Arkansas.csv
California.csv
Colorado.csv
Connecticut.csv
Delaware.csv
District of Columbia.csv
Florida.csv
Georgia.csv
Hawaii.csv
Idaho.csv
Illinois.csv
Indiana.csv
Iowa.csv



In [9]:
census_lookup = pd.read_csv("./export/census_tracts_lookup/California.csv", delimiter=",")

In [10]:
census_lookup.head()

,state_code,state,county_code,county,census_tracts,census_tract_number
0,6,California,1,Alameda County,3076,4301.01
1,6,California,1,Alameda County,230,4229.00
2,6,California,1,Alameda County,341,4041.02
3,6,California,1,Alameda County,1107,4371.02
4,6,California,1,Alameda County,2607,4273.00


In [11]:
import os

def make_sure_directory_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def export(path, filename, df, append=False):
    make_sure_directory_exists(path)
    filepath = os.path.join(path, filename)

    if not append:
        df.to_csv(filepath ,index=False, sep=',')
    else:
        df.to_csv(filepath, mode = 'a',index=False, sep=',')
   
''' These are the HDMA dataset columns'''
COLUMNS = [
    'action_taken',
    'action_taken_name',
    'agency_abbr',
    'agency_code',
    'agency_name',
    'applicant_ethnicity',
    'applicant_ethnicity_name',
    'applicant_income_000s',
    'applicant_race_1',
    'applicant_race_2',
    'applicant_race_3',
    'applicant_race_4',
    'applicant_race_5',
    'applicant_race_name_1',
    'applicant_race_name_2',
    'applicant_race_name_3',
    'applicant_race_name_4',
    'applicant_race_name_5',
    'applicant_sex',
    'applicant_sex_name',
    'application_date_indicator',
    'as_of_year',
    'census_tract_number',
    'co_applicant_ethnicity',
    'co_applicant_ethnicity_name',
    'co_applicant_race_1',
    'co_applicant_race_2',
    'co_applicant_race_3',
    'co_applicant_race_4',
    'co_applicant_race_5',
    'co_applicant_race_name_1',
    'co_applicant_race_name_2',
    'co_applicant_race_name_3',
    'co_applicant_race_name_4',
    'co_applicant_race_name_5',
    'co_applicant_sex',
    'co_applicant_sex_name',
    'county_code',
    'county_name',
    'denial_reason_1',
    'denial_reason_2',
    'denial_reason_3',
    'denial_reason_name_1',
    'denial_reason_name_2',
    'denial_reason_name_3',
    'edit_status',
    'edit_status_name',
    'hoepa_status',
    'hoepa_status_name',
    'hud_median_family_income',
    'lien_status',
    'lien_status_name',
    'loan_amount_000s',
    'loan_purpose',
    'loan_purpose_name',
    'loan_type',
    'loan_type_name',
    'minority_population',
    'msamd',
    'msamd_name',
    'number_of_1_to_4_family_units',
    'number_of_owner_occupied_units',
    'owner_occupancy',
    'owner_occupancy_name',
    'population',
    'preapproval',
    'preapproval_name',
    'property_type',
    'property_type_name',
    'purchaser_type',
    'purchaser_type_name',
    'rate_spread',
    'respondent_id',
    'sequence_number',
    'state_abbr',
    'state_code',
    'state_name',
    'tract_to_msamd_income'
]
    

In [ ]:
''' Queries the HDMA API for the census tracts specified in the census_lookup dataframe.'''
import requests

verbose = True
page_size = 250000
path = './export/hmda_data'

is_first_census = True
count = 0
for row in census_lookup.iterrows():
    count += 1
    state_code = row[1][0]
    state_name = row[1][1]
    county_code = '{:03d}'.format(int(row[1][2]))
    county_name = row[1][3]
    census_tracts= row[1][4]
    census_tract_number = row[1][5]
    
    results = []
    offset = 0
    url_template = 'https://api.consumerfinance.gov:443/data/hmda/slice/hmda_lar.json?$where=state_code={}+AND+county_code={}+AND+census_tract_number="{}"&$limit={}&$offset={}'
    while True:
        url = url_template.format(state_code, county_code, census_tract_number, page_size, offset)
        response = requests.get(url).json()
        if response is None or response.get('results') is None:
            continue
        for loan in response['results']:
            results.append([loan.get(column) for column in COLUMNS])
            
        if len(results) < int(response['total']):
            offset += page_size
        else:
            break
            
    # export after every census
    filename = '2016_{}_HDMA.csv'.format(state_name)
    filepath = os.path.join(path, filename)
    verbose and print("{:05d} :: saving {} to '{}'".format(count,census_tract_number, filepath))
    state_df = pd.DataFrame(results, dtype=str, columns = COLUMNS)
    export(path, filename, state_df, append=not is_first_census)        
    is_first_census = False
    
print('Finished')
    

00001 :: saving 4301.01 to './export/hmda_data/2016_California_HDMA.csv'
00002 :: saving 4229.0 to './export/hmda_data/2016_California_HDMA.csv'
00003 :: saving 4041.02 to './export/hmda_data/2016_California_HDMA.csv'
00004 :: saving 4371.02 to './export/hmda_data/2016_California_HDMA.csv'
00005 :: saving 4273.0 to './export/hmda_data/2016_California_HDMA.csv'
00006 :: saving 4283.01 to './export/hmda_data/2016_California_HDMA.csv'
00007 :: saving 4331.04 to './export/hmda_data/2016_California_HDMA.csv'
00008 :: saving 4285.0 to './export/hmda_data/2016_California_HDMA.csv'
00009 :: saving 4003.0 to './export/hmda_data/2016_California_HDMA.csv'
00010 :: saving 4362.0 to './export/hmda_data/2016_California_HDMA.csv'
00011 :: saving 4443.02 to './export/hmda_data/2016_California_HDMA.csv'
00012 :: saving 4321.0 to './export/hmda_data/2016_California_HDMA.csv'
00013 :: saving 4012.0 to './export/hmda_data/2016_California_HDMA.csv'
00014 :: saving 4058.0 to './export/hmda_data/2016_Califor

00115 :: saving 4028.0 to './export/hmda_data/2016_California_HDMA.csv'
00116 :: saving 4361.0 to './export/hmda_data/2016_California_HDMA.csv'
00117 :: saving 4301.02 to './export/hmda_data/2016_California_HDMA.csv'
00118 :: saving 4021.0 to './export/hmda_data/2016_California_HDMA.csv'
00119 :: saving 4019.0 to './export/hmda_data/2016_California_HDMA.csv'
00120 :: saving 4042.0 to './export/hmda_data/2016_California_HDMA.csv'
00121 :: saving 4356.01 to './export/hmda_data/2016_California_HDMA.csv'
00122 :: saving 4039.0 to './export/hmda_data/2016_California_HDMA.csv'
00123 :: saving 4054.02 to './export/hmda_data/2016_California_HDMA.csv'
00124 :: saving 4403.35 to './export/hmda_data/2016_California_HDMA.csv'
00125 :: saving 4037.01 to './export/hmda_data/2016_California_HDMA.csv'
00126 :: saving 4382.03 to './export/hmda_data/2016_California_HDMA.csv'
00127 :: saving 4503.0 to './export/hmda_data/2016_California_HDMA.csv'
00128 :: saving 4325.01 to './export/hmda_data/2016_Califo

00229 :: saving 4419.23 to './export/hmda_data/2016_California_HDMA.csv'
00230 :: saving 4085.0 to './export/hmda_data/2016_California_HDMA.csv'
00231 :: saving 4018.0 to './export/hmda_data/2016_California_HDMA.csv'
00232 :: saving 4415.01 to './export/hmda_data/2016_California_HDMA.csv'
00233 :: saving 4441.0 to './export/hmda_data/2016_California_HDMA.csv'
00234 :: saving 4101.0 to './export/hmda_data/2016_California_HDMA.csv'
00235 :: saving 4215.0 to './export/hmda_data/2016_California_HDMA.csv'
00236 :: saving 4035.02 to './export/hmda_data/2016_California_HDMA.csv'
00237 :: saving 4356.02 to './export/hmda_data/2016_California_HDMA.csv'
00238 :: saving 4090.0 to './export/hmda_data/2016_California_HDMA.csv'
00239 :: saving 4514.03 to './export/hmda_data/2016_California_HDMA.csv'
00240 :: saving 4231.0 to './export/hmda_data/2016_California_HDMA.csv'
00241 :: saving 4371.01 to './export/hmda_data/2016_California_HDMA.csv'
00242 :: saving 4331.02 to './export/hmda_data/2016_Califo

00343 :: saving 4506.07 to './export/hmda_data/2016_California_HDMA.csv'
00344 :: saving 4089.0 to './export/hmda_data/2016_California_HDMA.csv'
00345 :: saving 4337.0 to './export/hmda_data/2016_California_HDMA.csv'
00346 :: saving 4046.0 to './export/hmda_data/2016_California_HDMA.csv'
00347 :: saving 4274.0 to './export/hmda_data/2016_California_HDMA.csv'
00348 :: saving 4212.0 to './export/hmda_data/2016_California_HDMA.csv'
00349 :: saving 4366.02 to './export/hmda_data/2016_California_HDMA.csv'
00350 :: saving 4308.0 to './export/hmda_data/2016_California_HDMA.csv'
00351 :: saving 4412.0 to './export/hmda_data/2016_California_HDMA.csv'
00352 :: saving 4431.01 to './export/hmda_data/2016_California_HDMA.csv'
00353 :: saving 4357.0 to './export/hmda_data/2016_California_HDMA.csv'
00354 :: saving 4363.0 to './export/hmda_data/2016_California_HDMA.csv'
00355 :: saving 4384.0 to './export/hmda_data/2016_California_HDMA.csv'
00356 :: saving 4324.0 to './export/hmda_data/2016_California

00459 :: saving 22.0 to './export/hmda_data/2016_California_HDMA.csv'
00460 :: saving 33.0 to './export/hmda_data/2016_California_HDMA.csv'
00461 :: saving 3.0 to './export/hmda_data/2016_California_HDMA.csv'
00462 :: saving 1.01 to './export/hmda_data/2016_California_HDMA.csv'
00463 :: saving 5.01 to './export/hmda_data/2016_California_HDMA.csv'
00464 :: saving 36.0 to './export/hmda_data/2016_California_HDMA.csv'
00465 :: saving 5.02 to './export/hmda_data/2016_California_HDMA.csv'
00466 :: saving 7.0 to './export/hmda_data/2016_California_HDMA.csv'
00467 :: saving 21.0 to './export/hmda_data/2016_California_HDMA.csv'
00468 :: saving 13.0 to './export/hmda_data/2016_California_HDMA.csv'
00469 :: saving 18.0 to './export/hmda_data/2016_California_HDMA.csv'
00470 :: saving 27.0 to './export/hmda_data/2016_California_HDMA.csv'
00471 :: saving 12.0 to './export/hmda_data/2016_California_HDMA.csv'
00472 :: saving 31.0 to './export/hmda_data/2016_California_HDMA.csv'
00473 :: saving 35.0 t

...

02174 :: saving 2933.06 to './export/hmda_data/2016_California_HDMA.csv'
02175 :: saving 5358.03 to './export/hmda_data/2016_California_HDMA.csv'
02176 :: saving 2678.0 to './export/hmda_data/2016_California_HDMA.csv'
02177 :: saving 6026.0 to './export/hmda_data/2016_California_HDMA.csv'
02178 :: saving 2964.0 to './export/hmda_data/2016_California_HDMA.csv'
02179 :: saving 6504.0 to './export/hmda_data/2016_California_HDMA.csv'
02180 :: saving 2114.1 to './export/hmda_data/2016_California_HDMA.csv'
02181 :: saving 2111.1 to './export/hmda_data/2016_California_HDMA.csv'
02182 :: saving 4009.0 to './export/hmda_data/2016_California_HDMA.csv'
02183 :: saving 6702.01 to './export/hmda_data/2016_California_HDMA.csv'
02184 :: saving 9003.0 to './export/hmda_data/2016_California_HDMA.csv'
02185 :: saving 1905.1 to './export/hmda_data/2016_California_HDMA.csv'
02186 :: saving 4621.0 to './export/hmda_data/2016_California_HDMA.csv'
02187 :: saving 5356.07 to './export/hmda_data/2016_Californi

02287 :: saving 2145.0 to './export/hmda_data/2016_California_HDMA.csv'
02288 :: saving 6209.01 to './export/hmda_data/2016_California_HDMA.csv'
02289 :: saving 1901.0 to './export/hmda_data/2016_California_HDMA.csv'
02290 :: saving 2213.03 to './export/hmda_data/2016_California_HDMA.csv'
02291 :: saving 9107.11 to './export/hmda_data/2016_California_HDMA.csv'
02292 :: saving 5313.02 to './export/hmda_data/2016_California_HDMA.csv'
02293 :: saving 5041.02 to './export/hmda_data/2016_California_HDMA.csv'
02294 :: saving 9203.03 to './export/hmda_data/2016_California_HDMA.csv'
02295 :: saving 3018.01 to './export/hmda_data/2016_California_HDMA.csv'
02296 :: saving 1436.03 to './export/hmda_data/2016_California_HDMA.csv'
02297 :: saving 9800.3 to './export/hmda_data/2016_California_HDMA.csv'
02298 :: saving 1909.01 to './export/hmda_data/2016_California_HDMA.csv'
02299 :: saving 2673.0 to './export/hmda_data/2016_California_HDMA.csv'
02300 :: saving 2170.0 to './export/hmda_data/2016_Cali

02400 :: saving 1373.01 to './export/hmda_data/2016_California_HDMA.csv'
02401 :: saving 2397.01 to './export/hmda_data/2016_California_HDMA.csv'
02402 :: saving 3020.03 to './export/hmda_data/2016_California_HDMA.csv'
02403 :: saving 9110.0 to './export/hmda_data/2016_California_HDMA.csv'
02404 :: saving 2426.0 to './export/hmda_data/2016_California_HDMA.csv'
02405 :: saving 9202.0 to './export/hmda_data/2016_California_HDMA.csv'
02406 :: saving 5350.02 to './export/hmda_data/2016_California_HDMA.csv'
02407 :: saving 1918.1 to './export/hmda_data/2016_California_HDMA.csv'
02408 :: saving 5724.0 to './export/hmda_data/2016_California_HDMA.csv'
02409 :: saving 3105.01 to './export/hmda_data/2016_California_HDMA.csv'
02410 :: saving 6012.12 to './export/hmda_data/2016_California_HDMA.csv'
02411 :: saving 2132.02 to './export/hmda_data/2016_California_HDMA.csv'
02412 :: saving 1241.02 to './export/hmda_data/2016_California_HDMA.csv'
02413 :: saving 9800.19 to './export/hmda_data/2016_Cali

02513 :: saving 2971.2 to './export/hmda_data/2016_California_HDMA.csv'
02514 :: saving 5706.03 to './export/hmda_data/2016_California_HDMA.csv'
02515 :: saving 5521.0 to './export/hmda_data/2016_California_HDMA.csv'
02516 :: saving 4335.03 to './export/hmda_data/2016_California_HDMA.csv'
02517 :: saving 2214.02 to './export/hmda_data/2016_California_HDMA.csv'
02518 :: saving 4018.0 to './export/hmda_data/2016_California_HDMA.csv'
02519 :: saving 2264.2 to './export/hmda_data/2016_California_HDMA.csv'
02520 :: saving 2060.5 to './export/hmda_data/2016_California_HDMA.csv'
02521 :: saving 2094.01 to './export/hmda_data/2016_California_HDMA.csv'
02522 :: saving 6003.03 to './export/hmda_data/2016_California_HDMA.csv'
02523 :: saving 2674.04 to './export/hmda_data/2016_California_HDMA.csv'
02524 :: saving 5334.03 to './export/hmda_data/2016_California_HDMA.csv'
02525 :: saving 1892.02 to './export/hmda_data/2016_California_HDMA.csv'
02526 :: saving 4817.11 to './export/hmda_data/2016_Cali

02626 :: saving 2149.02 to './export/hmda_data/2016_California_HDMA.csv'
02627 :: saving 2244.1 to './export/hmda_data/2016_California_HDMA.csv'
02628 :: saving 1319.0 to './export/hmda_data/2016_California_HDMA.csv'
02629 :: saving 9201.04 to './export/hmda_data/2016_California_HDMA.csv'
02630 :: saving 6025.05 to './export/hmda_data/2016_California_HDMA.csv'
02631 :: saving 3022.01 to './export/hmda_data/2016_California_HDMA.csv'
02632 :: saving 6512.22 to './export/hmda_data/2016_California_HDMA.csv'
